In [1]:
import numpy as np
import pandas as pd
import polars as pl
import sys
from natsort import natsorted
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
pd.set_option('display.max_columns',None)
import psycopg2
#to scale the data using z-score 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
#Algorithms to use
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
#Metrics to evaluate the model
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve
import warnings
warnings.filterwarnings("ignore")
#importing PCA and TSNE
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [37]:
df = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Positions_extensions/Latest_data/WG_100Kb_Bins.xlsx')
df.columns = ['chrom', 'start', 'end']
print(df.shape)
df.head(3)

(27267, 3)


,chrom,start,end
0,chr1,0,100000
1,chr1,100000,200000
2,chr1,200000,300000


In [38]:
data = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Positions_extensions/Latest_data/main_data_new_02_02_2024.xlsx')
print(data.shape)
data.head()

(14438, 4)


,chrom,start,end,type
0,chr1,3010218,3087058,cLAD
1,chr1,3126730,3196670,cLAD
2,chr1,3220610,3282860,cLAD
3,chr1,3339298,3357024,cLAD
4,chr1,3371740,3389850,cLAD


In [39]:
data[data['start'] == 102912878]

,chrom,start,end,type
5867,chr3,102912878,105173833,ciLAD


In [109]:
dataset1 = pd.DataFrame(df)
dataset2 = pd.DataFrame(data)

# Initialize an empty result DataFrame to store the matching regions
result_df = pd.DataFrame(columns=['chrom', 'start', 'end', 'start_df1', 'end_df1'])

# Iterate over each row in dataset2
for index2, row2 in dataset2.iterrows():
    # Find the matching rows in dataset1 based on chrom and start/end conditions
    matching_rows = dataset1[(dataset1['chrom'] == row2['chrom']) & ((dataset1['start'] <= row2['start']) & (row2['start'] <= dataset1['end']) | (dataset1['start'] <= row2['end']) & (row2['end'] <= dataset1['end']))]
    
    # Create new rows in the result DataFrame for each match
    for index1, matching_row in matching_rows.iterrows():
        result_df = result_df.append({'chrom': matching_row['chrom'],
                                      'start': row2['start'],
                                      'end': row2['end'],
                                      'start_df1': matching_row['start'],
                                      'end_df1': matching_row['end']},
                                     ignore_index=True)
        
        # Check if there is a gap between the matching_row and the current row in dataset2
        gap_start = matching_row['end']
        gap_end = row2['start']
        gap_size = gap_end - gap_start

        # If the gap is less than 100000, create a new row in the result DataFrame
        if gap_size < 100000:
            result_df = result_df.append({'chrom': matching_row['chrom'],
                                          'start': gap_start,
                                          'end': gap_end,
                                          'start_df1': matching_row['start'],
                                          'end_df1': matching_row['end']},
                                         ignore_index=True)

# Display the result DataFrame
result_df

,chrom,start,end,start_df1,end_df1
0,chr1,3010218,3087058,3000000,3100000
1,chr1,3100000,3010218,3000000,3100000
2,chr1,3087059,3126729,3000000,3100000
3,chr1,3100000,3087059,3000000,3100000
4,chr1,3087059,3126729,3100000,3200000
...,...,...,...,...,...
40323,chrX,166200000,166142678,166100000,166200000
40324,chrX,166163391,166206469,166100000,166200000
40325,chrX,166200000,166163391,166100000,166200000
40326,chrX,166163391,166206469,166200000,166300000


In [40]:
dataset1 = pd.DataFrame(df)
dataset2 = pd.DataFrame(data)

# Initialize an empty result DataFrame to store the matching regions
result_df = pd.DataFrame(columns=['chrom', 'start', 'end', 'start_df1', 'end_df1'])

# Iterate over each row in dataset2
for index2, row2 in dataset2.iterrows():
    # Find the matching rows in dataset1 based on chrom and start/end conditions
    matching_rows = dataset1[(dataset1['chrom'] == row2['chrom']) & ((dataset1['start'] <= row2['start']) & (row2['start'] <= dataset1['end']) | (dataset1['start'] <= row2['end']) & (row2['end'] <= dataset1['end']))]
    
    # Create new rows in the result DataFrame for each match
    for index1, matching_row in matching_rows.iterrows():
        result_df = result_df.append({'chrom': matching_row['chrom'],
                                      'start': row2['start'],
                                      'end': row2['end'],
                                      'start_df1': matching_row['start'],
                                      'end_df1': matching_row['end']},
                                     ignore_index=True)

# Display the result DataFrame
result_df

,chrom,start,end,start_df1,end_df1
0,chr1,3010218,3087058,3000000,3100000
1,chr1,3126730,3196670,3100000,3200000
2,chr1,3220610,3282860,3200000,3300000
3,chr1,3339298,3357024,3300000,3400000
4,chr1,3371740,3389850,3300000,3400000
...,...,...,...,...,...
22801,chrX,164898980,164923418,164900000,165000000
22802,chrX,164928258,165004632,164900000,165000000
22803,chrX,164928258,165004632,165000000,165100000
22804,chrX,165323428,165367578,165300000,165400000


In [41]:
result_df[result_df['start'] == 102912878]

,chrom,start,end,start_df1,end_df1
9614,chr3,102912878,105173833,102900000,103000000
9615,chr3,102912878,105173833,105100000,105200000


In [42]:
main = pd.merge(result_df, data, on = ['chrom', 'start', 'end'], how = 'left', sort=False)
main = main[['chrom', 'start', 'end', 'type', 'start_df1', 'end_df1']]
main

,chrom,start,end,type,start_df1,end_df1
0,chr1,3010218,3087058,cLAD,3000000,3100000
1,chr1,3126730,3196670,cLAD,3100000,3200000
2,chr1,3220610,3282860,cLAD,3200000,3300000
3,chr1,3339298,3357024,cLAD,3300000,3400000
4,chr1,3371740,3389850,cLAD,3300000,3400000
...,...,...,...,...,...,...
22801,chrX,164898980,164923418,fLAD,164900000,165000000
22802,chrX,164928258,165004632,fLAD,164900000,165000000
22803,chrX,164928258,165004632,fLAD,165000000,165100000
22804,chrX,165323428,165367578,fLAD,165300000,165400000


In [43]:
main['Difference'] = main['end'] - main['start']
main

,chrom,start,end,type,start_df1,end_df1,Difference
0,chr1,3010218,3087058,cLAD,3000000,3100000,76840
1,chr1,3126730,3196670,cLAD,3100000,3200000,69940
2,chr1,3220610,3282860,cLAD,3200000,3300000,62250
3,chr1,3339298,3357024,cLAD,3300000,3400000,17726
4,chr1,3371740,3389850,cLAD,3300000,3400000,18110
...,...,...,...,...,...,...,...
22801,chrX,164898980,164923418,fLAD,164900000,165000000,24438
22802,chrX,164928258,165004632,fLAD,164900000,165000000,76374
22803,chrX,164928258,165004632,fLAD,165000000,165100000,76374
22804,chrX,165323428,165367578,fLAD,165300000,165400000,44150


In [44]:
main[main['start'] == 102912878]

,chrom,start,end,type,start_df1,end_df1,Difference
9614,chr3,102912878,105173833,ciLAD,102900000,103000000,2260955
9615,chr3,102912878,105173833,ciLAD,105100000,105200000,2260955


In [45]:
main['concat_new'] = main['chrom'].astype('str') + '_' + main['start_df1'].astype('str') + '_' + main['end_df1'].astype('str')
main

,chrom,start,end,type,start_df1,end_df1,Difference,concat_new
0,chr1,3010218,3087058,cLAD,3000000,3100000,76840,chr1_3000000_3100000
1,chr1,3126730,3196670,cLAD,3100000,3200000,69940,chr1_3100000_3200000
2,chr1,3220610,3282860,cLAD,3200000,3300000,62250,chr1_3200000_3300000
3,chr1,3339298,3357024,cLAD,3300000,3400000,17726,chr1_3300000_3400000
4,chr1,3371740,3389850,cLAD,3300000,3400000,18110,chr1_3300000_3400000
...,...,...,...,...,...,...,...,...
22801,chrX,164898980,164923418,fLAD,164900000,165000000,24438,chrX_164900000_165000000
22802,chrX,164928258,165004632,fLAD,164900000,165000000,76374,chrX_164900000_165000000
22803,chrX,164928258,165004632,fLAD,165000000,165100000,76374,chrX_165000000_165100000
22804,chrX,165323428,165367578,fLAD,165300000,165400000,44150,chrX_165300000_165400000


In [59]:
df.to_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Positions_extensions/Latest_data/updated_distance02_02_2024.xlsx',index=False)

In [60]:
df_new = main.copy()

In [47]:
df_new[df_new['start'] == 102912878]

,chrom,start,end,type,start_df1,end_df1,Difference,concat_new
9614,chr3,102912878,105173833,ciLAD,102900000,103000000,2260955,chr3_102900000_103000000
9615,chr3,102912878,105173833,ciLAD,105100000,105200000,2260955,chr3_105100000_105200000


In [119]:
df_new.to_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Positions_extensions/Latest_data/main_final_new_dummy_data.xlsx', index=False)

In [5]:
import pandas as pd
df_new = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Positions_extensions/Latest_data/mapped_result_df1_df2_logic.xlsx')
df_new

,chrom,start,end,type,Difference,start_df1,end_df1,concat,concat_new
0,chr1,3010218,3087058,cLAD,76840,3000000,3100000,3000000_3100000,chr1_3000000_3100000
1,chr1,3087059,3126729,fLAD,39670,3000000,3100000,3000000_3100000,chr1_3000000_3100000
2,chr1,3087059,3126729,fLAD,39670,3100000,3200000,3100000_3200000,chr1_3100000_3200000
3,chr1,3126730,3196670,cLAD,69940,3100000,3200000,3100000_3200000,chr1_3100000_3200000
4,chr1,3196671,3220609,fLAD,23938,3100000,3200000,3100000_3200000,chr1_3100000_3200000
...,...,...,...,...,...,...,...,...,...
20159,chrX,165367579,166142677,cLAD,775098,165300000,165400000,165300000_165400000,chrX_165300000_165400000
20160,chrX,165367579,166142677,cLAD,775098,166100000,166200000,166100000_166200000,chrX_166100000_166200000
20161,chrX,166142678,166163390,fLAD,20712,166100000,166200000,166100000_166200000,chrX_166100000_166200000
20162,chrX,166163391,166206469,ciLAD,43078,166100000,166200000,166100000_166200000,chrX_166100000_166200000


In [61]:
df = df_new.copy()
df['difference'] = df.apply(lambda row: len(range(max(row['start'], row['start_df1']), min(row['end'], row['end_df1']))), axis=1)
df

,chrom,start,end,type,start_df1,end_df1,Difference,concat_new,difference
0,chr1,3010218,3087058,cLAD,3000000,3100000,76840,chr1_3000000_3100000,76840
1,chr1,3126730,3196670,cLAD,3100000,3200000,69940,chr1_3100000_3200000,69940
2,chr1,3220610,3282860,cLAD,3200000,3300000,62250,chr1_3200000_3300000,62250
3,chr1,3339298,3357024,cLAD,3300000,3400000,17726,chr1_3300000_3400000,17726
4,chr1,3371740,3389850,cLAD,3300000,3400000,18110,chr1_3300000_3400000,18110
...,...,...,...,...,...,...,...,...,...
22801,chrX,164898980,164923418,fLAD,164900000,165000000,24438,chrX_164900000_165000000,23418
22802,chrX,164928258,165004632,fLAD,164900000,165000000,76374,chrX_164900000_165000000,71742
22803,chrX,164928258,165004632,fLAD,165000000,165100000,76374,chrX_165000000_165100000,4632
22804,chrX,165323428,165367578,fLAD,165300000,165400000,44150,chrX_165300000_165400000,44150


In [9]:
df_new.to_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Positions_extensions/Latest_data/new_preclimax_data_02_02_2024.xlsx', index=False)

In [49]:
df_new = df.copy()
df_new['concat_new'] = pd.Categorical(df_new['concat_new'])
df_new = df_new.loc[df_new.groupby('concat_new')['difference'].idxmax()]
df_new

,chrom,start,end,type,start_df1,end_df1,Difference,concat_new,difference
13066,chr10,100044008,100183876,fLAD,100000000,100100000,139868,chr10_100000000_100100000,55992
13067,chr10,100044008,100183876,fLAD,100100000,100200000,139868,chr10_100100000_100200000,83876
13068,chr10,100443718,100508304,fLAD,100400000,100500000,64586,chr10_100400000_100500000,56282
765,chr10,100508305,100564187,cLAD,100500000,100600000,55882,chr10_100500000_100600000,55882
13071,chr10,100625046,100724954,fLAD,100600000,100700000,99908,chr10_100600000_100700000,74954
...,...,...,...,...,...,...,...,...,...
22497,chrX,98591751,98886751,fLAD,98800000,98900000,295000,chrX_98800000_98900000,86751
22498,chrX,99613243,99790011,fLAD,99600000,99700000,176768,chrX_99600000_99700000,86757
22499,chrX,99613243,99790011,fLAD,99700000,99800000,176768,chrX_99700000_99800000,90011
22501,chrX,99797273,99893911,fLAD,99800000,99900000,96638,chrX_99800000_99900000,93911


In [50]:
df = df_new.copy()

# Function to impute missing rows
def impute_missing_rows(df):
    result = pd.DataFrame(columns=df.columns)
    
    for chrom, group in df.groupby('chrom'):
        start = group['start_df1'].min()
        end = group['end_df1'].max()
        
        # Generate new rows based on 100000 difference
        new_rows = pd.DataFrame({
            'start_df1': range(start, end, 100000),
            'end_df1': range(start + 100000, end + 100000, 100000),
            'chrom': [chrom]*len(range(start, end, 100000))
        })
        
        result = pd.concat([result, new_rows], ignore_index=True)
    
    return result

# Concatenate the original DataFrame with the imputed rows
result_df = pd.concat([df, impute_missing_rows(df)], ignore_index=True)

# Sort the DataFrame based on 'chrom' and 'start_df1'
result_df = result_df.sort_values(by=['chrom', 'start_df1']).reset_index(drop=True)
result_df.drop_duplicates(subset=['start_df1', 'end_df1', 'chrom'], inplace=True)
result_df

,chrom,start,end,type,start_df1,end_df1,Difference,concat_new,difference
0,chr1,3010218,3087058,cLAD,3000000,3100000,76840,chr1_3000000_3100000,76840
2,chr1,3126730,3196670,cLAD,3100000,3200000,69940,chr1_3100000_3200000,69940
4,chr1,3220610,3282860,cLAD,3200000,3300000,62250,chr1_3200000_3300000,62250
6,chr1,3282861,3339297,fLAD,3300000,3400000,56436,chr1_3300000_3400000,39297
8,chr1,3420830,3664260,cLAD,3400000,3500000,243430,chr1_3400000_3500000,79170
...,...,...,...,...,...,...,...,...,...
33818,chrX,NaN,NaN,NaN,165800000,165900000,NaN,NaN,NaN
33819,chrX,NaN,NaN,NaN,165900000,166000000,NaN,NaN,NaN
33820,chrX,NaN,NaN,NaN,166000000,166100000,NaN,NaN,NaN
33821,chrX,165367579,166142677,cLAD,166100000,166200000,775098,chrX_166100000_166200000,42677


In [51]:
df = result_df.copy()

In [52]:
cols = ['start', 'end', 'type', 'Difference']
df.loc[:,cols] = df.loc[:,cols].ffill()
df

,chrom,start,end,type,start_df1,end_df1,Difference,concat_new,difference
0,chr1,3010218,3087058,cLAD,3000000,3100000,76840,chr1_3000000_3100000,76840
2,chr1,3126730,3196670,cLAD,3100000,3200000,69940,chr1_3100000_3200000,69940
4,chr1,3220610,3282860,cLAD,3200000,3300000,62250,chr1_3200000_3300000,62250
6,chr1,3282861,3339297,fLAD,3300000,3400000,56436,chr1_3300000_3400000,39297
8,chr1,3420830,3664260,cLAD,3400000,3500000,243430,chr1_3400000_3500000,79170
...,...,...,...,...,...,...,...,...,...
33818,chrX,165323428,165367578,fLAD,165800000,165900000,44150,NaN,NaN
33819,chrX,165323428,165367578,fLAD,165900000,166000000,44150,NaN,NaN
33820,chrX,165323428,165367578,fLAD,166000000,166100000,44150,NaN,NaN
33821,chrX,165367579,166142677,cLAD,166100000,166200000,775098,chrX_166100000_166200000,42677


In [53]:
df_final = df[['chrom', 'start_df1', 'end_df1', 'type']]
#df_final.to_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/Positions_extensions/Latest_data/cLAD_ciLAD_fLAD_Final_modified_02_02_2024.bed', sep='\t',header=None, index=False)
df_final

,chrom,start_df1,end_df1,type
0,chr1,3000000,3100000,cLAD
2,chr1,3100000,3200000,cLAD
4,chr1,3200000,3300000,cLAD
6,chr1,3300000,3400000,fLAD
8,chr1,3400000,3500000,cLAD
...,...,...,...,...
33818,chrX,165800000,165900000,fLAD
33819,chrX,165900000,166000000,fLAD
33820,chrX,166000000,166100000,fLAD
33821,chrX,166100000,166200000,cLAD


In [54]:
df_final.to_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Positions_extensions/Latest_data/madhu_02_02_2024_updated.xlsx', index=False)

In [55]:
df_final = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Positions_extensions/Latest_data/madhu_02_02_2024_updated.xlsx')
df_final.to_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/Positions_extensions/Latest_data/Updated_cLAD_ciLAD_fLAD_Final_modified_02_02_2024.bed', sep='\t',header=None, index=False)
df_final

,chrom,start_df1,end_df1,type
0,chr1,3000000,3100000,cLAD
1,chr1,3100000,3200000,cLAD
2,chr1,3200000,3300000,cLAD
3,chr1,3300000,3400000,fLAD
4,chr1,3400000,3500000,cLAD
...,...,...,...,...
25432,chrX,165800000,165900000,fLAD
25433,chrX,165900000,166000000,fLAD
25434,chrX,166000000,166100000,fLAD
25435,chrX,166100000,166200000,cLAD


In [80]:
result = pd.concat([df1, df2, df3], ignore_index=True)
result_sorted = result.sort_values(by=['chrom', 'start'])
result_sorted

,chrom,start,end,type
0,chr1,3010218,3087058,cLAD
7229,chr1,3087059,3126729,fLAD
1,chr1,3126730,3196670,cLAD
7230,chr1,3196671,3220609,fLAD
2,chr1,3220610,3282860,cLAD
...,...,...,...,...
3805,chrX,165004633,165323427,cLAD
14436,chrX,165323428,165367578,fLAD
3806,chrX,165367579,166142677,cLAD
14437,chrX,166142678,166163390,fLAD


In [84]:
pip install natsort


  Obtaining dependency information for natsort from https://files.pythonhosted.org/packages/ef/82/7a9d0550484a62c6da82858ee9419f3dd1ccc9aa1c26a1e43da3ecd20b0d/natsort-8.4.0-py3-none-any.whl.metadata



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [85]:
from natsort import natsorted
result_sorted['chrom'] = natsorted(result_sorted['chrom'], key=lambda x: [(int(text) if text.isdigit() else text.lower()) for text in re.split('([0-9]+)', x)])
result_sorted = result_sorted.sort_values(by=['chrom', 'start'])
result_sorted

,chrom,start,end,type
0,chr1,3010218,3087058,cLAD
7229,chr1,3087059,3126729,fLAD
1,chr1,3126730,3196670,cLAD
7230,chr1,3196671,3220609,fLAD
2,chr1,3220610,3282860,cLAD
...,...,...,...,...
3805,chrX,165004633,165323427,cLAD
14436,chrX,165323428,165367578,fLAD
3806,chrX,165367579,166142677,cLAD
14437,chrX,166142678,166163390,fLAD


In [10]:
df_final.to_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Positions_extensions/Latest_data/madhu.xlsx', index=False)

In [79]:
result_sorted = result.sort_values(by=['chrom', 'start'])
result_sorted

,chrom,start,end,type
0,chr1,3010218,3087058,cLAD
7229,chr1,3087059,3126729,fLAD
1,chr1,3126730,3196670,cLAD
7230,chr1,3196671,3220609,fLAD
2,chr1,3220610,3282860,cLAD
...,...,...,...,...
3805,chrX,165004633,165323427,cLAD
14436,chrX,165323428,165367578,fLAD
3806,chrX,165367579,166142677,cLAD
14437,chrX,166142678,166163390,fLAD


In [12]:
df1 = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/Positions_extensions/Latest_data/cLAD_MM10_Version.bed', sep='\t', header=None)
df1.columns = ['chrom', 'start', 'end', 'count']
df1['type'] = 'cLAD'
df1 = df1[['chrom', 'start', 'end', 'type']] 
df1.head(3)

,chrom,start,end,type
0,chr1,3010218,3087058,cLAD
1,chr1,3126730,3196670,cLAD
2,chr1,3220610,3282860,cLAD


In [17]:
df2[df2['start'] == 102912878]

,chrom,start,end,type
2060,chr3,102912878,105173833,ciLAD


In [14]:
df2 = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/Positions_extensions/Latest_data/ciLAD_MM10_Version.bed', sep='\t', header=None)
df2.columns = ['chrom', 'start', 'end', 'count']
df2['type'] = 'ciLAD'
df2 = df2[['chrom', 'start', 'end', 'type']] 
df2.head(3)

,chrom,start,end,type
0,chr1,4753810,4790980,ciLAD
1,chr1,4803510,4939730,ciLAD
2,chr1,5003180,5034298,ciLAD


In [15]:
df3 = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/Positions_extensions/Latest_data/fLAD_MM10_Version.bed', sep='\t', header=None)
df3.columns = ['chrom', 'start', 'end', 'count']
df3['type'] = 'fLAD'
df3 = df3[['chrom', 'start', 'end', 'type']] 
df3.head(3)

,chrom,start,end,type
0,chr1,3087059,3126729,fLAD
1,chr1,3196671,3220609,fLAD
2,chr1,3282861,3339297,fLAD


In [16]:
result = pd.concat([df1, df2, df3], keys=['df1', 'df2', 'df3'])

# If you want to reset the index
result.reset_index(drop=True, inplace=True)
result

,chrom,start,end,type
0,chr1,3010218,3087058,cLAD
1,chr1,3126730,3196670,cLAD
2,chr1,3220610,3282860,cLAD
3,chr1,3339298,3357024,cLAD
4,chr1,3371740,3389850,cLAD
...,...,...,...,...
14433,chrX,164633150,164643508,fLAD
14434,chrX,164898980,164923418,fLAD
14435,chrX,164928258,165004632,fLAD
14436,chrX,165323428,165367578,fLAD


In [21]:
result.to_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Positions_extensions/Latest_data/main_data_new_02_02_2024.xlsx', index=False)

In [15]:
import pandas as pd
# Load BED file into a pandas DataFrame
df = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/Positions_extensions/fLAD_MM10_100kb_bins.bed', sep='\t', header=None)
df.columns = ['chrom', 'start', 'end']
# Function to split rows if the difference is not 100,000
def split_rows(row):
    start = row['start']
    end = row['end']
    chrom = row['chrom']
    
    diff = end - start
    
    if diff == 100000:
        return row
    else:
        num_splits = diff // 100000
        rows = []
        for i in range(num_splits):
            new_start = start + i * 100000
            new_end = new_start + 100000
            new_row = {'start': new_start, 'end': new_end, 'chrom': chrom}
            rows.append(new_row)
        return pd.DataFrame(rows)

# Apply the function to each row and concatenate the results
result_df = pd.concat([split_rows(row) if row['end'] - row['start'] != 100000 else pd.DataFrame([row]) for index, row in df.iterrows()], ignore_index=True)
result_df.to_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/Positions_extensions/fLAD_MM10_100kb_bins_updated.bed', sep='\t',header=None, index=False)
result_df

,chrom,start,end
0,chr1,4900000,5000000
1,chr1,5100000,5200000
2,chr1,5200000,5300000
3,chr1,6100000,6200000
4,chr1,6200000,6300000
...,...,...,...
7092,chr18,164100000,164200000
7093,chr18,164200000,164300000
7094,chr18,164300000,164400000
7095,chr18,164500000,164600000


In [16]:
import pandas as pd
# Load BED file into a pandas DataFrame
df = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/Positions_extensions/fLAD_MM10_100kb_bins.bed', sep='\t', header=None)
df.columns = ['chrom', 'start', 'end']
df

,chrom,start,end
0,chr1,4900000,5000000
1,chr1,5100000,5200000
2,chr1,5200000,5300000
3,chr1,6100000,6200000
4,chr1,6200000,6300000
...,...,...,...
2895,chr18,163000000,163900000
2896,chr18,164100000,164300000
2897,chr18,164300000,164400000
2898,chr18,164500000,164600000


In [17]:
result_df[result_df['start'] == 163000000]

,chrom,start,end
7083,chr18,163000000,163100000


In [27]:
df = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/Positions_extensions/merged_updated.bed', sep='\t', header=None)
df

,0,1,2,3
0,chr1,3000000,3100000,cLAD
1,chr1,3100000,3200000,cLAD
2,chr1,3200000,3300000,cLAD
3,chr1,3400000,3500000,cLAD
4,chr1,3500000,3600000,cLAD
...,...,...,...,...
23748,chr18,164100000,164200000,fLAD
23749,chr18,164200000,164300000,fLAD
23750,chr18,164300000,164400000,fLAD
23751,chr18,164500000,164600000,fLAD


In [28]:
df_sorted = df.sort_values(by=[0, 1])
df_sorted.columns = ['chrom', 'start', 'end', 'type']

In [20]:
df_sorted.to_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/Positions_extensions/new_merged_updated.bed', sep='\t',header=None, index=False)

In [29]:
df_sorted.columns = ['chrom', 'start', 'end', 'type']
df_sorted

,chrom,start,end,type
0,chr1,3000000,3100000,cLAD
1,chr1,3100000,3200000,cLAD
9139,chr1,3100000,3200000,ciLAD
2,chr1,3200000,3300000,cLAD
9140,chr1,3200000,3300000,ciLAD
...,...,...,...,...
8218,chr3,165600000,165700000,cLAD
8219,chr3,165700000,165800000,cLAD
8220,chr3,165800000,165900000,cLAD
8221,chr3,165900000,166000000,cLAD


In [30]:
df_sorted.type.value_counts()

ciLAD    8433
cLAD     8223
fLAD     7097
Name: type, dtype: int64

In [31]:
df_sorted['concat'] = df_sorted['chrom'].astype(str) + '_' + df_sorted['start'].astype(str) + '_' + df_sorted['end'].astype(str)
df_sorted

,chrom,start,end,type,concat
0,chr1,3000000,3100000,cLAD,chr1_3000000_3100000
1,chr1,3100000,3200000,cLAD,chr1_3100000_3200000
9139,chr1,3100000,3200000,ciLAD,chr1_3100000_3200000
2,chr1,3200000,3300000,cLAD,chr1_3200000_3300000
9140,chr1,3200000,3300000,ciLAD,chr1_3200000_3300000
...,...,...,...,...,...
8218,chr3,165600000,165700000,cLAD,chr3_165600000_165700000
8219,chr3,165700000,165800000,cLAD,chr3_165700000_165800000
8220,chr3,165800000,165900000,cLAD,chr3_165800000_165900000
8221,chr3,165900000,166000000,cLAD,chr3_165900000_166000000


In [32]:
df_sorted = df_sorted[['concat', 'type']]
df_sorted

,concat,type
0,chr1_3000000_3100000,cLAD
1,chr1_3100000_3200000,cLAD
9139,chr1_3100000_3200000,ciLAD
2,chr1_3200000_3300000,cLAD
9140,chr1_3200000_3300000,ciLAD
...,...,...
8218,chr3_165600000_165700000,cLAD
8219,chr3_165700000_165800000,cLAD
8220,chr3_165800000_165900000,cLAD
8221,chr3_165900000_166000000,cLAD


In [50]:
df = df_sorted.copy()

In [54]:
df_sorted[df_sorted['concat'] == 'chr1_3800000_3900000']

,concat,type
7,chr1_3800000_3900000,cLAD
9146,chr1_3800000_3900000,ciLAD
17214,chr1_3800000_3900000,fLAD


In [59]:
df_sorted[df_sorted['concat'] == 'chr1_3900000_4000000']

,concat,type
8,chr1_3900000_4000000,cLAD
17215,chr1_3900000_4000000,fLAD


In [60]:
df_sorted[df_sorted['concat'] == 'chr1_4000000_4100000']

,concat,type
9,chr1_4000000_4100000,cLAD
9147,chr1_4000000_4100000,ciLAD
17216,chr1_4000000_4100000,fLAD


In [58]:
df[df['concat'] == 'chr1_3800000_3900000']

,chrom,start,end,type,concat
11917,chr1,3800000,3900000,ciLAD,chr1_3800000_3900000


In [51]:
def prioritize_types(types):
    if 'ciLAD' in types.values:
        return 'ciLAD'
    elif 'cLAD' in types.values:
        return 'cLAD'
    elif 'fLAD' in types.values:
        return 'fLAD'
    elif 'cLAD' in types.values and 'ciLAD' in types.values:
        return 'ciLAD'
    elif 'ciLAD' in types.values and 'fLAD' in types.values:
        return 'ciLAD'
    elif 'cLAD' in types.values and 'fLAD' in types.values:
        return 'cLAD'
    elif 'cLAD' in types.values and 'ciLAD' in types.values and 'fLAD' in types.values:
        return 'ciLAD'

df = df.groupby('concat')['type'].agg(prioritize_types).reset_index()
df

,concat,type
0,chr10_100000000_100100000,fLAD
1,chr10_10000000_10100000,cLAD
2,chr10_100100000_100200000,ciLAD
3,chr10_100200000_100300000,ciLAD
4,chr10_100300000_100400000,ciLAD
...,...,...
14265,chr3_9800000_9900000,cLAD
14266,chr3_98400000_98500000,cLAD
14267,chr3_98500000_98600000,cLAD
14268,chr3_9900000_10000000,cLAD


In [52]:
df_sorted[df_sorted['concat'] == 'chr10_10000000_10100000']

,concat,type
1124,chr10_10000000_10100000,cLAD
17828,chr10_10000000_10100000,fLAD
18254,chr10_10000000_10100000,fLAD
18688,chr10_10000000_10100000,fLAD


In [53]:
df['chrom'] = df['concat'].str.split('_').str[0]
df['start'] = df['concat'].str.split('_').str[1]
df['end'] = df['concat'].str.split('_').str[2]
df = df[['chrom', 'start', 'end', 'type']]
df.sort_values(by=['chrom', 'start'], inplace=True)
df.to_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/Positions_extensions/new_merged_sorted.bed', sep='\t',header=None, index=False)
df

,chrom,start,end,type
10739,chr1,10000000,10100000,ciLAD
10738,chr1,100000000,100100000,cLAD
10740,chr1,100100000,100200000,cLAD
10741,chr1,100200000,100300000,cLAD
10742,chr1,100300000,100400000,cLAD
...,...,...,...,...
14264,chr3,98000000,98100000,cLAD
14266,chr3,98400000,98500000,cLAD
14267,chr3,98500000,98600000,cLAD
14268,chr3,9900000,10000000,cLAD


In [57]:
df['concat'] = df['chrom'].astype(str) + '_' + df['start'].astype(str) + '_' + df['end'].astype(str)


In [11]:
result_df.difference.value_counts()

100000    8223
Name: difference, dtype: int64

In [10]:
result_df['difference'] = result_df['end'] - result_df['start']
result_df

,chrom,start,end,difference
0,chr1,3000000,3100000,100000
1,chr1,3100000,3200000,100000
2,chr1,3200000,3300000,100000
3,chr1,3400000,3500000,100000
4,chr1,3500000,3600000,100000
...,...,...,...,...
8218,chr3,165600000,165700000,100000
8219,chr3,165700000,165800000,100000
8220,chr3,165800000,165900000,100000
8221,chr3,165900000,166000000,100000


In [9]:
# Function to split rows if the difference is not 100,000
def split_rows(row):
    start = row['start']
    end = row['end']
    chrom = row['chrom']
    
    diff = end - start
    
    if diff == 100000:
        return row
    else:
        num_splits = diff // 100000
        rows = []
        for i in range(num_splits):
            new_start = start + i * 100000
            new_end = new_start + 100000
            new_row = {'start': new_start, 'end': new_end, 'chrom': chrom}
            rows.append(new_row)
        return pd.DataFrame(rows)

# Apply the function to each row and concatenate the results
result_df = pd.concat([split_rows(row) if row['end'] - row['start'] != 100000 else pd.DataFrame([row]) for index, row in df.iterrows()], ignore_index=True)
result_df

,chrom,start,end
0,chr1,3000000,3100000
1,chr1,3100000,3200000
2,chr1,3200000,3300000
3,chr1,3400000,3500000
4,chr1,3500000,3600000
...,...,...,...
8218,chr3,165600000,165700000
8219,chr3,165700000,165800000
8220,chr3,165800000,165900000
8221,chr3,165900000,166000000
